System found in **Subharmonic transitions and Bloch-Siegert shift in electrically driven spin resonance**: 10.1103/PhysRevB.92.054422 

In [1]:
# Objects for defining the Hamiltonian
from SymPT import *
import sympy as sp

In [2]:
omega0 = RDSymbol('omega0', real=True, positive=True)
omega = RDSymbol('omega', real=True, positive=True, order=1)
Eac = RDSymbol(r'\tilde{E}_{\mathrm{ac}}', real=True, order=1)
B = RDSymbol(r'\tilde{B}', real=True, order=1)
alpha = RDSymbol(r'\tilde{\alpha}', real=True, order=1)
theta = RDSymbol(r'\theta', real=True)

nso = sp.Matrix([0, sp.cos(theta), sp.sin(theta)])

Spin = RDBasis('sigma', 2)
s0, sx, sy, sz = Spin.basis
s_vec = sp.Matrix([sx, sy, sz])

a = BosonOp('a')
ad = Dagger(a)

H0 = hbar * omega0 * (ad*a + sp.Rational(1,2))
HE = Eac * sp.sin(omega * t) * (ad + a)
HB = -sp.Rational(1,2) * B * sz
Hso = I * alpha * (ad - a) * nso.dot(s_vec)

display_dict({
    sp.Symbol('H_0'): H0,
    sp.Symbol('H_E'): HE,
    sp.Symbol('H_B'): HB,
    sp.Symbol(r'H_{\mathrm{so}}'): Hso,
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
Eff_Frame = EffectiveFrame(H=H0 + HB, V=HE + Hso, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.


# Standard Schrieffer-Wolff Transformation

In [4]:
Eff_Frame.solve(max_order=5, method="SW")
H_eff = Eff_Frame.get_H('matrix')
H2_corrections = Eff_Frame.corrections

Performing Time Dependent SWT for each order: 100%|██████████| 5/5 [00:23<00:00,  4.80s/it]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to matrix form: 100%|██████████| 6/6 [00:00<00:00, 33.43it/s]


In [28]:
# Corrections to the Hamiltonian for each order
H0, H1, H2, H3, H4, H5 = [H_correction for H_correction in Eff_Frame.corrections.values()]

In [29]:
Spin.project(H0).expand()

hbar*omega0*Dagger(a)*a*sigma_0 + hbar*omega0*sigma_0/2

In [30]:
Spin.project(H1).expand()

-\tilde{B}*sigma_3/2

In [31]:
Spin.project(sp.expand_complex(H2).trigsimp()).expand().cancel()

-(\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2*sigma_0 + \tilde{\alpha}**2*sigma_0)/(hbar*omega0)

In [32]:
H3_ = Spin.project(sp.expand_complex(H3.expand().subs({ad:0, a:0})).trigsimp())
display_dict({k:sp.factor_terms(v.cancel()) for k,v in group_by_operators(H3_).items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [37]:
H4_ = Spin.project(sp.expand_complex(H4.expand().subs({ad:0, a:0})).trigsimp())
display_dict(group_by_operators(H4_))

<IPython.core.display.Math object>

In [38]:
H5 = Spin.project(sp.expand_complex(H5.expand().subs({ad:0, a:0}).expand()).trigsimp())
display_dict(group_by_operators(H5))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [39]:
group_by_operators(H5)[sx].factor()

-\tilde{B}*\tilde{E}_{\mathrm{ac}}*\tilde{\alpha}*(\tilde{B}**2 - 2*\tilde{\alpha}**2)*sin(omega*t)*cos(\theta)/(hbar**4*omega0**4)

In [40]:
group_by_operators(H5)[sy].subs(sp.cos(theta)*sp.sin(theta), sp.Rational(1,2)*sp.sin(2*theta)).collect(B*alpha*sp.sin(2*theta))

\tilde{B}*\tilde{\alpha}*(-\tilde{B}**2*\tilde{\alpha}/(2*hbar**4*omega0**4) - \tilde{E}_{\mathrm{ac}}**2*\tilde{\alpha}*sin(omega*t)**2/(2*hbar**4*omega0**4) + \tilde{\alpha}**3/(2*hbar**4*omega0**4))*sin(2*\theta) - \tilde{E}_{\mathrm{ac}}*\tilde{\alpha}*omega**3*cos(\theta)*cos(omega*t)/(hbar*omega0**4)

In [41]:
group_by_operators(H5)[sz].collect(B*alpha*sp.cos(theta)**2)

\tilde{B}*\tilde{\alpha}*(\tilde{B}**2*\tilde{\alpha}/(hbar**4*omega0**4) - \tilde{E}_{\mathrm{ac}}**2*\tilde{\alpha}*cos(omega*t)**2/(hbar**4*omega0**4) + \tilde{E}_{\mathrm{ac}}**2*\tilde{\alpha}/(hbar**4*omega0**4) - \tilde{\alpha}**3/(hbar**4*omega0**4))*cos(\theta)**2 - \tilde{E}_{\mathrm{ac}}*\tilde{\alpha}*omega**3*sin(\theta)*cos(omega*t)/(hbar*omega0**4)